In [ ]:
import networkx as nx
from neo4j import GraphDatabase

In [ ]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"

In [ ]:
def fetch_graph_data( tx ):
    query = """
    MATCH (n)-[r]->(m)
    RETURN 
    
        elementId(n) AS source_id,
        labels(n) AS source_labels,
        properties(n) AS source_properties, 
        
        elementId(m) AS target_id, 
        labels(m) AS target_labels,
        properties(m) AS target_properties, 
        
        type(r) AS edge_type
    """
    return list( tx.run( query ) )

In [ ]:
# Initialize a directed NetworkX graph
G = nx.MultiDiGraph()

# Initialize the driver
driver = GraphDatabase.driver( uri, auth=(username, password) )
# Retrieve data from Neo4j and populate the NetworkX graph
with driver.session() as session:
    results = session.execute_read( fetch_graph_data )

    for record in results:
        
        source_id = record[ "source_id" ]
        source_labels = record[ "source_labels" ]
        source_properties = record[ "source_properties" ]
        target_id = record[ "target_id" ]
        target_labels = record[ "target_labels" ]
        target_properties = record[ "target_properties" ]
        relationship_type = record[ "edge_type" ]

        G.add_node( source_id, labels=source_labels, **source_properties )
        G.add_node( target_id, labels=target_labels, **target_properties )

        # Add the edge with relationship type as an edge attribute
        G.add_edge( source_id, target_id, type=relationship_type )

driver.close()

In [ ]:
print( "Nodes:", G.nodes( data=True ) )
print( "Edges:", G.edges( data=True ) )

In [ ]:
import matplotlib.pyplot as plt

plt.figure( figsize=(10, 10) )
nx.draw( G, with_labels=True, node_size=500, node_color="lightblue", font_size=10 )
plt.show()